In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

plt.rcParams["font.family"] = ["SimHei", "WenQuanYi Micro Hei", "Heiti TC"]

df = pd.read_csv('../../data/intermediate/初步清洗_比赛数据.csv', encoding='utf-8')

print(df['政治面貌'].value_counts())

# 清洗数据
# 使用字典方式替换政治面貌列中的值
df['政治面貌'] = df['政治面貌'].replace({
    '团员': '共青团员',
    '学生': '群众',
    '入党积极分子': '共青团员'
})
print('清洗后：\n')
print(df['政治面貌'].value_counts())

# 查看不同政治面貌在思政课满意度上的均值
satisfaction_cols = ['思政课总体满意度', '思政课设置满意度', '思政课内容满意度', 
                     '思政课质量满意度', '思政课效果满意度']

print("\n不同政治面貌群体的思政课满意度均值:")
for col in satisfaction_cols:
    print(f"\n{col}:")
    print(df.groupby('政治面貌')[col].mean())


In [ ]:
# 计算各政治面貌群体的满意度均值
satisfaction_by_political_affiliation = df.groupby('政治面貌')[satisfaction_cols].mean()

# 1. 政治面貌分布饼图
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
political_counts = df['政治面貌'].value_counts()
plt.pie(political_counts.values, labels=political_counts.index, autopct='%1.1f%%', startangle=90)
plt.title('不同政治面貌人群分布')

# 2. 各项满意度对比柱状图
plt.subplot(1, 2, 2)
x = np.arange(len(satisfaction_cols))
width = 0.25

plt.bar(x - width, satisfaction_by_political_affiliation.loc['中共党员（含预备党员）'],
        width, label='中共党员（含预备党员）')
plt.bar(x, satisfaction_by_political_affiliation.loc['共青团员'],
        width, label='共青团员')
plt.bar(x + width, satisfaction_by_political_affiliation.loc['群众'],
        width, label='群众')

plt.xlabel('思政课满意度维度')
plt.ylabel('满意度均值')
plt.title('不同政治面貌群体的思政课满意度对比')
plt.xticks(x, [col.replace('思政课', '').replace('满意度', '') for col in satisfaction_cols], rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

# 3. 热力图展示详细数据
plt.figure(figsize=(10, 6))
sns.heatmap(satisfaction_by_political_affiliation, annot=True, cmap='YlGnBu', fmt='.3f')
plt.title('不同政治面貌群体思政课满意度热力图')
plt.xlabel('满意度维度')
plt.ylabel('政治面貌')
plt.tight_layout()
plt.show()

# 4. 差异分析雷达图
plt.figure(figsize=(8, 8))
categories = [col.replace('思政课', '').replace('满意度', '') for col in satisfaction_cols]

# 计算相对于平均值的差异
avg_satisfaction = df[satisfaction_cols].mean()
diff_data = satisfaction_by_political_affiliation.sub(avg_satisfaction, axis=1)

angles = np.linspace(0, 2 * np.pi, len(categories), endpoint=False).tolist()
angles += angles[:1]  # 闭合图形

fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))

for political_status in diff_data.index:
    values = diff_data.loc[political_status].tolist()
    values += values[:1]  # 闭合图形
    ax.plot(angles, values, 'o-', linewidth=2, label=political_status)
    ax.fill(angles, values, alpha=0.25)

ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories)
ax.set_ylim(-0.01, 0.015)
plt.title('不同政治面貌群体满意度与总体平均值的差异')
plt.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
plt.tight_layout()
plt.show()

# 输出具体数值
print("各群体满意度均值:")
print(satisfaction_by_political_affiliation)
print("\n与总体平均的差异:")
print(diff_data)

# 这个挖掘点不是很合适，建议废弃